# Chapter 10. 스파크 SQL

## 10.1 SQL이란

SQL 또는 구조적 질의 언어(Structured Query Language)는 데이터에 대한 관계형 연산을 표현하기 위한 도메인 특화 언어.

## 10.2 빅데이터와 SQL : 아파치 하이브

스파크가 등장하기 전 하이브가 빅데이터 SQL 접근 계층에서 사실상의 표준이었다.

스파크는 RDD를 이용하는 범용 처리 엔진으로 시작했지만 이제는 많은 사용자가 스파크 SQL을 사용하고 있다.

## 10.3 빅데이터와 SQL : 스파크 SQL

스파크 2.0 버전에는 하이브를 지원할 수 있는 상위 호환 기능으로 ANSI-SQL과 HiveQL을 모두 지원하는 자체 개발된 SQL 파서가 포함.

쓰리프트 서버나 SQL 인터페이스에 접속해 스파크의 연산 능력 활용 가능.

데이터 엔지니와 과학자는 전체 데이터 처리 파이프라인에 스파크 SQL 사용 가능.

통합형 API는 SQL로 데이터를 조회하고 DataFrame으로 변환한 다음 스파크의 MLlib이 제공하는 대규모 머신러닝 알고리즘 중 하나를 사용해 수팽한 결과를 다른 데이터소스에 저장하는 전체 과정을 가능하게 함.

### 10.3.1 스파크와 하이브의 관계

스파크 SQL은 하이브 메타스토어를 사용하므로 하이브와 잘 연동 가능.

스파크 SQL은 하이브 메타스토어에 접속(단, 이미 하이브를 사용하고 있는 경우)한 뒤 조회할 파일 수를 최소화하기 위해 메타데이터를 참조.

**하이브 메타스토어**

하이브 메타스토어에 접속하려면 접근하려는 하이브 메타스토어에 적합한 버전을 spark.sql.hive.metastore.version에 설정. (기본값은 1.2.1)

HiveMetastoreClient가 초기화되는 방식을 변경하려면 spark.sql.hive.metastore.jars 설정.

하이브 메타스토어ㅓ가 저장된 다른 데이터베이스에 접속하려면 적합한 클래스 접두사 정의해야 함.

스파크와 하이브에서 공유할 수 있도록 클래스 접두사를 spark.sql.hive.metastore.sharedPrefixes 속성에 설정.

## 10.4 스파크 SQL 쿼리 실행 방법

스파크는 SQL 쿼리를 실행할 수 있는 몇 가지 인터페이스 제공.

### 10.4.1 스파크 SQL CLI

스파크 SQL CLI(Command Line Interface, 명령행 인터페이스)는 로컬 환경의 명령행에서 기본 스파크 SQL 쿼리를 실행할 수 있는 편리한 도구.

스파크 SQL CLI는 쓰리프트 JDBC 서버와 통신할 수 없음.

* 스파크 SQL CLI를 사용하기 위한 명령

> ./bin/spark-sql

* 스파크가 설치된 경로의 conf 디렉터리에 hive-site.xml, core-site.xml, hdfs-site.xml 파일을 배치해 하이브를 사용할 수 있는 환경 구성 가능.

* 사용 가능한 전체 옵션을 보기위한 명령

> ./bin/spark-sql --help

### 10.4.2 스파크의 프로그래밍 SQL 인터페이스

서버를 설정해 SQL을 사용할 수도 있지만, SparkSession 객체의 sql 메서드를 사용해 스파크에서 지원하는 언어 API로 비정형 SQL을 실행하는 것도 가능.

처리된 결과는 DataFrame 반환.

In [0]:
# 예시 

spark.sql("SELECT 1 + 1").show()

spark.sql("SELECT 1+1") 명령은 프로그래밍 방식으로 평가할 수 있는 DataFrame을 반환.

다른 트랜스포메이션과 마찬가지로 즉시 실행되지 않고 지연 처리.

함수에 여러 줄로 구성된 문자열을 전달할 수 있으므로 여러 줄로 구성된 쿼리를 아주 간단히 표현 가능.

In [0]:
# 예시) 파이썬이나 스칼라에서 사용 가능

spark.sql("""SELECT user_id, department, first_name FROM professors
  WHERE department IN
    (SELECT name department WHERE created_date >= '2016-01-01')""")

In [0]:
# DataFrame을 생성해 SQL을 사용해 처리하고 그 결과를 다시 DataFrame으로 반환

spark.read.format("json").load("dbfs:/FileStore/shared_uploads/hyjeong0815@gmail.com/2015_summary-1.json")\
  .createOrReplaceTempView("some_sql_view")  # DataFrame을 SQL에서 사용할 수 있도록 처리

spark.sql("""
  SELECT DEST_COUNTRY_NAME, sum(cost)
  FROM some_sql_view GROUP BY DEST_COUNTRY_NAME""")\
  .where("DEST_COUNTRY_NAME like 'S%'").where("'sum(count)' > 10")\
  .count()

### 10.4.3 스파크 SQL 쓰리프트 JDBC/ODBC 서버

스파크는 자바 데이터베이스 연결 인터페이스를 제공.

* JDBC/ODBC 서버를 시작하기 위한 명령

> ./sbin/start-thriftserver.sh  # bin/spark-submit 스크립트에서 사용할 수 있는 모든 명령행 옵션 지원

* 쓰리프트 서버의 전체 설정 옵션을 확인

> ./sbin/start-thriftserver.sh --helf

쓰리프트 서버는 기본적으로 localhost:10000 주소 사용하는데 환경 변수나 시스템 속성을 지정해 쓰리프트 서버의 주소 변경 가능.

* 환경변수 설정

> export HIVE_SERVER2_THRIFT_PORT=< listening-port >

  >  export HIVE_SERVER2_THRIFT_BIND_HOST=< listening-host >

  >  ./sbin/start-thriftserver.sh \

  >    --master < master-uri > \

  >    ...
  

  >  ./sbin/start-threiftserver.sh \

  >    --hiveconf hive.server2.thrift.port=< listening-port > \

  >    --hiveconf hive.server2.thrift.host=< listening-host > \

  >    --master < master-uri >

  >    ...
  
* 서버가 시작되면 접속 테스트 실행 명령

> ./bin/beeline

> beline> !connect jdbc:hive2://localhost:10000

beline은 사용자 이름과 비밀번호를 요구하는데, 비보안 모드의 경우 단순히 로컬 사용자 이름을 입력하고 비밀번호는 입력하지 않아도 된다.

반면에 보안 모드의 경우 beeline 문서에서 제시하는 방법을 따라야 함. (https://bit.ly/2N8dLy9)

## 10.5 카탈로그

스파크 SQL에서 가장 높은 추상화 단계.

카탈로그는 테이블에 저장된 데이터에 대한 메타데이터뿐만 아니라 데이터베이스, 테이블, 함수 그리고 뷰에 대한 정보 추상화.

* 카탈로그 사용 패키지

> org.apache.spark.sql.catalog.Catalog

카탈로그는 테이블, 데이터베이스 그리고 함수를 조회하는 등 여러 가지 유용한 함수를 제공.

## 10.6 테이블

스파크 SQL을 사용해 유용한 작업을 수행하려면 먼저 테이블을 정의해야 함.

테이블은 명령을 시행할 데이터의 구조라는 점에서 DataFrame과 논리적으로 동일하지만 DataFrame은 프로그래밍 언어로 정의되는 반면에 테이블은 데이터베이스에서 정의.

스파크에서 테이블을 생성하면 default 데이터베이스에 등록.

(스파크 2.x 버전에서 테이블은 항상 데이터를 가지고 있다. 임시 테이블의 개념이 없으며 데이터를 갖지 않은 뷰만 존재. 따라서 테이블을 제거하면 모든 데이터 제거)

### 10.6.1 스파크 관리형 테이블

테이블은 테이블의 데이터와 테이블에 대한 데이터, 즉 메타데이터 이 두 가지 중요한 정보 저장.

디스크에 저장된 파일을 이용해 테이블을 정의하면 외부 테이블을 정의하는 것.

DataFrame의 saveAsTable 메서드는 스파크가 관련된 모든 정보를 추적할 수 있는 **관리형 테이블** 생성.

saveAsTable 메서드는 테이블을 읽고 데이터를 스파크 포맷으로 변환한 후 새로운 경로에 저장해 새로운 실행 계획에 이러한 동작이 반영되어 있음을 알 수 있으며, 하이브의 기본 웨어하우스 경로에 데이터를 저장하는 것을 확인 가능.

* 데이터 저장 경로 변경

> spark.sql.warehouse.dir 속성에 원하는 디렉터리 경로 설정 (기본 저장 경로 : /user/hive/warehouse)

### 10.6.2 테이블 생성하기

스파크 SQL에서는 전체 데이터소스 API를 재사용할 수 있는 독특한 기능 지원. 

즉, 테이블을 정의한 다음 테이블에 데이터를 적재할 필요가 없다.

> CREATE TABLE flights(

>  DEST_COUNTRY_NAME STRING,

 >  ORIGIN_COUNTRY_NAME STRING,
  
  > count LONG)

> USING JSON OPTIONS (path '/data/flight-data/json/2015-summary.json')

포맷을 지정하지 않으면 스파크는 기본적으로 하이브 SerDe 설정을 사용하는데, 하이브 SerDe는 스파크의 자체 직렬화보다 훨씬 느리므로 테이블을 사용하는 Reader와 Writer 성능에 영향을 미친다. 따라서 USING 구문은 중요.

하이브 사용자는 STORED AS 구문으로 하이브 테이블 생성 가능.

* 테이블의 특정 컬럼에 코멘트를 추가해 다른 개발자의 이해를 도울 수 있음.

> CREATE TABLE flights_csv(

> DEST_COUNTRY_NAME STRIN,

> ORIGIN_COUNTRY_NAME STRING COMMENT "remember", the US will be most prevalent",

> count LONG)

> USING csv OPTIOINS (header true, path '/data/flight-data/csv/2015-summary.csv')

* SELECT쿼리의 실행 결과를 이용해 테이블 생성 가능.

> REATE TABLE flights_from_select USING parquet AS SELECT * FROM flights

* 테이블이 없는 경우에만 생성하도록 지정 가능.


* 파티ㅕ닝된 데이터셋을 저장해 데이터 레이아웃 제어 가능

> CREATE TABLE partitioned_flights USING parquet PARTITIONED BY (DEST_COUNTRY_NAME)

> AS SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 5

* 스파크에 접속한 세션에서도 생성된 테이블 사용 가능.

### 10.6.3 외부 테이블 생성하기

* 예제 데이터에 미리 저장된 데이터 파일의 내용을 테이블 형태로 볼 수 있음

> CREATE EXTERNAL TABLE hive_flights (

> DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)

> ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION '/data/flight-data-hive/'

> CREATE EXTERMINAL TABLE hive_flights_2

> ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

> LOCATION '/data/flight-data-hive/' AS SELECT * FROM flights

### 10.6.4 테이블에 데이터 삽입하기

데이터 삽입은 표준 SQL 문법을 따름.

> INSERT INTO flights_from_select 

> SELECT DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count FROM flights LIMIT 20

특정 파티션에만 저장하고 싶은 경우 파티션 명세를 추가하는 것도 가능.

쓰기 연산은 파티셔닝 스키아메 맞게 데이터 저장.

> INSERT INTO partitioned_fligts

> PARTITION (DEST_COUNTRY_NAME='UNITED STATES')

> SELECT count, ORIGIN_COUNTRY_NAME FROM flights

> WHERE DEST_COUNTRY_NAME='UNITED STATES' LIMIT 12

### 10.6.5 테이블 메타데이터 확인하기

테이블 생성 시 코메트 추가 가능

> DESCRIBE TABLE flights_csv (테이블의 메타데이터 정보 반환)

파티셔닝 스키마 정보 확인 가능

> SHOW PARTITIONS partitioned_flights (단, 파티션된 테이블에서만 동작)

### 10.6.6 테이블 메타데이터 갱신하기

테이블 메타데이터를 유지하는 것은 가장 취신의 데이터셋을 읽고 있다는 것을 보장할 수 있는 중요한 작업.

* REFRESH TABLE 구문 - 테이블과 관련된 모든 캐싱된 항목 갱신

> REFRESH table partitioned_flights (테이블이 이미 캐싱되어 있다면 다음번 스캔이 동작하는 시점에 다시 캐싱)

* REPAIR TABLE 구문 - 카탈로그에서 관리하는 테이블의 파티션 정보를 새로 고침

> MSCK REPAIR TABLE partitioned_flights (새로운 파티션 정보를 수집하는데 초점을 맞추기 때문에 수동으로 신규 파티션을 만든다면 테이블을 repair해야 함.)

### 10.6.7 테이블 제거하기

테이블은 삭제할 수 없고, 제거만 가능하다. 

* 관리형 테이블을 제거하면 데이터와 테이블 정의 모두 제거

> DROP TABLE flight_csv;

* 테이블이 존재하는 경우에만 제거

> DROP TABLE IF EXISTS flight_csv;

**외부 테이블 제거하기**

외부 테이블을 제거하면 데이터는 삭제되지 않지만 외부 테이블명을 이용해 데이터를 조회할 수 없음.

### 10.6.8 테이블 캐싱하기

> CACHE TABLE flights

* 캐시에서 제거

> UNCACHE TABLE FLIGHTS

## 10.7 뷰

뷰는 기존 테이블에 여러 트랜스포메이션 작업을 지정. 

기본적으로 뷰는 단순 쿼리 실행 계획.

뷰를 사용해 쿼리 로직을 체계회하거나 재사용하기 편하게 만들 수 있다.

뷰는 데이터베이스에 설정하는 전역 뷰나 세션별 뷰가 될 수 있다.

### 10.7.1 뷰 생성하기

최종 사용자에게 뷰는 테이블처럼 보임. 

신규 경로에 모든 데이터를 다시 저장하는 대신 단순하게 쿼리 시점에 데이터소스에 fliter, select, 대규모 GROUP BY, ROLLUP 같은 트랜스포메이션 수행.

In [0]:
# 목적지가 United States인 항공운항 데이터를 보기 위한 뷰 생성

CREATE VIEW just_usa_view AS
  SELECT * FROM flights WHERE dest_country_name = 'United States'

In [0]:
# 데이터베이스에 등록하지 않고 현재 세션에만 사용할 수 있는 임시 뷰 생성

CREATE TEMP VIEW just_usa_view_temp AS
  SELECT * FROM flights WHERE dest_country_name = 'United States'

In [0]:
# 전역적인 임시 뷰 생성 - 데이터베이스에 상관없이 사용할 수 있으므로 전체 스파크 애플리케이션에서 볼 수 있지만 세션 종료시 사라짐.

CREATE GLOBAL TEMP VIEW just_usa_global_view_temp AS
  SELECT * FROM flights WHERE dest_country_name = 'United States'
  
SHOW TABLES

In [0]:
# 키워드를 사용해 생성된 뷰 덮어쓰기 (임시 뷰와 일반 뷰 모두 가능)

CREATE OR REPLACE TEMP VIEW just_usa_view_temp AS
  SELECT * FROM flights WHERE dest_country_name = 'United States'

In [0]:
# 다른 테이블과 동일한 방식으로 뷰 사용 가능

SELECT * FROM just_usa_view_temp

뷰는 실질적으로 트랜스포메이션이며 스파크는 쿼리가 실행될 때만 뷰에 정의된 트랜스포메이션 수행.

즉, 테이블의 데이터를 실제로 조회하는 경우에만 필터를 적용.

### 10.7.2 뷰 제거하기

테이블을 제거하는 것과 동일한 방식으로 뷰 제거 가능.

단지 제거하려는 대상을 테이블이 아닌 뷰로 지정.

단, 뷰는 어떤 데이터도 제거되지 않으며 뷰 정의만 제거

> DROP VIEW IF EXISTS just_usa_view;

## 10.8 데이터베이스

데이터베이스는 여러 테이블을 조직화하기 위한 도구.

스파크에서 실행하는 모든 SQL 명령문은 사용 중인 데이터베이스 범위에서 실행.

즉, 데이터베이스를 변경하면 이전에 생성된 모든 사용자 테이블이 변경되기 전의 데이터베이스에 속해 있으므로 기존 테이블 데이터를 조회하려면 다르게 쿼리해야 ㅏㅁ.

* 전체 데이터베이스 목록 확인

> SHOW DATABASES

### 10.8.1 데이터베이스 생성하기

> CREATE DATABASE some_db

### 10.8.2 데이터베이스 설정하기

USE 키워드 다음에 데이터베이스명을 붙여 쿼리 수행에 필요한 데이터베이스를 설정 가능.

> USE some_db

모든 쿼리는 테이블 이름을 찾을 때 앞서 지정한 데이터베이스를 참조. 하지만 다른 데이터베이스를 지정했기 때문에 정상 동작하던 쿼리가 실패하거나 다른 결과 반환할 수 있음.

> SHOW tables

> SELECT * FROM flights -- 테이블이나 뷰를 찾을 수 없어 에러 발생

올바른 접두사를 이용해 다른 데이터베이스의 테이블에 쿼리 수행 가능

> SELECT * FROM default.flights

현재 어떤 데이터베이스를 사용 중인지 확인

> SELECT current_databases()

기본 데이터베이스로 돌아가는 명령

> USE default;

### 10.8.3 데이터베이스 제거하기

> DROP DATABASE IF EXISTS some_db;

## 10.9 select 구문

스파크 쿼리는 ANSI-SQL 요건을 충족.

SELECT 표현식의 구조 확인 가능.

### 10.9.1 case...when...then 구문

SQL 쿼리의 값을 조건에 맞게 변경해야 하는 경우.

프로그래밍의 if 구문과 동일한 처리

In [0]:
SELECT 
  CASE WHEN DEST_COUNTRY_NAME = 'UNITED STATES' THEN 1
       WHEN DEST_COUNTRY_NAME = 'Egypt' THEN 0
       ELSE -1 END
FROM partitioned_flights

## 10.10 고급 주제

데이터를 쿼리하는 방법. 

SQL 구문은 조작, 정의, 제어와 관련된 명령 정의 가능.

### 10.10.1 복합 데이터 타입

복합 데이터 타입은 표준 SQL과 거리가 있으며 표준 SQL에는 존재하지 않는 기능.

**구조체**

맵에 더 가까우며 스파크에서 중첩 데이터를 생성하거나 쿼리하는 방법 제공.

* 구조체 생성

> CREATE VIEW IF NOT EXISTS nested_data AS

> SELECT (DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME) as country, count FROM flights

* 구조체 데이터 조회

> SELECT * FROM nested_data

* 구조체의 개별 컬럼 조회

> SELECT country.DEST_COUNTRY_NAME, count FROM nested_data

혹은 

> SELECT country.* , count FROM nested_data

**리스트**

값의 리스트를 만드는 collect_list 함수나 중복 값이 없는 배열을 만드는 collect_set 함수 사용

단, 두 함수 모두 집계 함수로 집계 연산 시에만 사용 가능.

> SELECT DEST_COUNTRY_NAME as new_name, collect_list(count) as flight_counts, collect_set(ORIGIN_COUNTRY_NAME) as origin_set

> FROM flights GROUP BY DEST_COUNTRY_NAME

* 컬럼에 직접 배열 생성

> SELECT DEST_COUNTRY_NAME, ARRAY(1, 2, 3) FROM flights

* 리스트의 특정 위치의 데이터 쿼리

> SELECT DEST_COUNTRY_NAME as new_name, collect_list(count)[0]

> FROM flights GROUP BY DEST_COUNTRY_NAME

* 배열을 다시 로우로 변환

> SEECT explode(collected_counts), DEST_COUNTRY NAME

> FROM flights_agg

### 10.10.2 함수

스파크 SQL은 복합 데이터 타입 외에도 다양한 고급 함수 제공.

* 스파크 SQL이 제공하는 전체 함수 목록 확인

> SHOW FUNCTIONS

* 스파크에 내장된 시스템 함수나 사용자 함수 중 어떤 유형의 함수 목록을 명확히 지정

> SHOW SYSTEM FUNCTIONS

* 사용자 정의 함수 확인

> SHOW USER FUNCTIONS

* 결과 필터링1 (* 키워드)

> SHOW FUNCTIONS "s*" # s로 시작하는 모든 함수 필터링

* 결과 필터링2 (LIKE 키워드)

> SHOW FUNCTIONS LIKE "collect*"

**사용자 정의 함수**

사용자 정의 함수를 제공하고 분산 환경에서 사용할 수 있는 기능 제공. 특정 언어를 사용해 함수를 개발한 다음 등록하여 함수 정의 가능.

In [0]:
def power3(number:Double):Double = number * number * number
  
spark.udf.register("power3", power3(_:Double):Double)

SELECT count, power3(count) FROM flights

# 하이브의 CREATE TEMPORARY FUNCTION 구문을 사용해도 함수 등록 가능

### 10.10.3 서브쿼리

서브쿼리를 사용하면 쿼리 안에 쿼리 지정 가능. 

**상호 연관 서브쿼리** 

: 서브쿼리의 정보를 보완하기 위해 쿼리의 외부 범위에 있는 일부 정보 사용

**비상호연관 서브쿼리**

: 외부 범위에 있는 정보를 사용하지 않음